In [3]:
import os
import numpy as np
import seaborn as sns
import pandas as pd
import subprocess
from dtw import correlate
import json
import librosa
import IPython.display as ipd
import math
import csv

MAX_AUDIO_DURATION_MS = 4 * 1000
MAX_SAMPLE_COMPARISON_COUNT = 1000

apppath = os.path.expanduser(os.getcwd() + '/')
mediapath = os.path.expanduser('/mnt/freesound-crawler/')
processedpath = os.path.join(mediapath, 'processed/')
soxipath = os.path.join(processedpath, 'soxi/')
wavpath = os.path.join(processedpath, 'wav/')
audiopath = os.path.join(processedpath, 'audio/')
normalizedpath = os.path.join(processedpath, 'normalized/')
dtwpath = os.path.join(processedpath, 'dtw/')
excludedpath = os.path.join(processedpath, 'excluded/')
extensionpath = os.path.join(apppath, 'Extension/data/')
extensionsamplepath = os.path.join(extensionpath, 'samples/')
librarypath = os.path.join(mediapath, 'downloads/')
metadatapath = os.path.join(mediapath, 'metadata/')
metadatacsvpath = os.path.join(processedpath, 'metadata.csv')
allowedfileformats = ['wav', 'aiff', 'flac', 'ogg', 'mp3', 'm4a']
audiofileformat = 'm4a'

for p in [processedpath, soxipath, wavpath, audiopath, normalizedpath, dtwpath, excludedpath, extensionpath, extensionsamplepath]:
    if not os.path.exists(p):
        os.makedirs(p)

In [4]:
ids = [f for f in os.listdir(metadatapath)]
print(len(ids))

if not os.path.exists(metadatacsvpath):
    fieldnames = ['id', 'duration', 'downloads', 'rating', 'numratings', 'tags']
    rows = []

    for id in ids:        
        with open(metadatapath + "/" + id) as file:
            try:
                id = int(id)
                line = file.readlines()[0]
                arguments = line.split('|')
                duration = int(arguments[0].split('duration:')[1])
                downloads = int(arguments[1].split('downloads:')[1])
                rating = int(arguments[2].split('rating:')[1])
                numratings = int(arguments[3].split('numratings:')[1])
                tags = arguments[4].split('tags:')[1]
                row = {
                    'id': id,
                    'duration': duration,
                    'downloads': downloads,
                    'rating': rating,
                    'numratings': numratings,
                    'tags': tags
                }
                rows.append(row)
            except Exception as e:
                print(e)

    with open(metadatacsvpath, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames = fieldnames)
        writer.writeheader()
        writer.writerows(rows)
        
def read_csv(filename):
    with open(filename) as f:
        file_data = csv.reader(f)
        headers = next(file_data)
        return [dict(zip(headers, i)) for i in file_data] 

metadata = read_csv(metadatacsvpath)

68455


In [5]:
excluded = [f for f in os.listdir(excludedpath)]
excluded = [f for f in excluded if f.endswith(audiofileformat)]
excluded = [f.split('.' + audiofileformat)[0] for f in excluded]

print(len(excluded))

974


In [6]:
DOWNLOADS_THRESHOLD = 25
RATING_THRESHOLD = 80
MIN_NUMRATINGS = 4

scored = []
for sample in metadata:            
    id = sample['id']
    if id in excluded:
        continue

    duration = int(sample['duration'])
    downloads = int(sample['downloads'])
    rating = int(sample['rating'])
    numratings = int(sample['numratings'])
    tags = sample['tags']

    if len(tags) > 0 and downloads >= DOWNLOADS_THRESHOLD and rating >= RATING_THRESHOLD and numratings >= MIN_NUMRATINGS:
        row = [id, duration, downloads, rating, numratings, tags, rating * downloads]
        scored.append(row)

print(len(scored))

scored.sort(key=lambda x: int(x[6]))
scored.reverse()

print(scored[0])

2496
['60009', 319, 67854, 86, 561, 'zoom-h2,domain,public,punch,swhish,swish,swoosh,swosh,whip,whoosh,woosh,stick,swash,zoom,swing,wind,wisch,attack,bamboo,cut,wish,flup,h2,luft,weapon,public-domain,air', 5835444]


In [7]:
samples = {}

for row in scored:
    id = row[0]
    samples[id] = {
        'duration': row[1],
        'downloads': row[2],
        'rating': row[3],
        'numratings': row[4],
        'tags': row[5],
        'score': row[6],
    }
    
print(samples[scored[0][0]])

{'duration': 319, 'downloads': 67854, 'rating': 86, 'numratings': 561, 'tags': 'zoom-h2,domain,public,punch,swhish,swish,swoosh,swosh,whip,whoosh,woosh,stick,swash,zoom,swing,wind,wisch,attack,bamboo,cut,wish,flup,h2,luft,weapon,public-domain,air', 'score': 5835444}


In [8]:
def get_fileformat(filename):
    return filename.split('.')[len(filename.split('.')) - 1]

all_filenames = [f for f in os.listdir(librarypath)]
fileformats = {}
for filename in all_filenames:
    fileformat = get_fileformat(filename)
    
    if fileformat not in fileformats:
        fileformats[fileformat] = 0
         
    fileformats[fileformat] = fileformats[fileformat] + 1
    
print('fileformats', fileformats)
print('all_filenames:', len(all_filenames))

processedfilenames = [f for f in os.listdir(audiopath) if f.endswith(audiofileformat)]

filenames = [f for f in os.listdir(librarypath) if f.endswith(tuple(allowedfileformats))]
filenames = [f for f in filenames if f.count('__') == 2]
filtered = []

for filename in filenames:
    id = filename.split('__')[0]
    
    for row in scored:
        if row[0] == id:
            filtered.append(filename)
filenames = filtered
print('filtered filenames', len(filenames))

fileformats {'flac': 2888, 'wav': 46250, 'mp3': 2936, 'aiff': 19174, 'ogg': 679, 'm4a': 142, 'crdownload': 15, 'DS_Store': 1}
all_filenames: 72085
filtered filenames 2492


In [9]:
def create_or_read_file(command, outfilename):
    if os.path.exists(outfilename):
        f = open(outfilename, "r")
        out = ''.join(f.readlines())
        f.close()
        return out
    else:
        try:
            out = str(subprocess.check_output(command).decode(encoding="utf-8")).strip().replace('\\n', '').replace("'", "")

            with open(outfilename, 'w') as file:
                file.write(out)

            return out
        except Exception as e:
            return 

In [10]:
def process(command, cwd="/"):
    subprocess.check_output(command, cwd=cwd)

res = []
for filename in filenames:
    try:
        fileartist = filename.split("__")[1].split("__")[0]
        fileid = filename.split('__')[0]
        audiofilename = fileid + '.' + audiofileformat
        
        if samples[fileid]['duration'] > MAX_AUDIO_DURATION_MS:
            continue

        if audiofilename not in processedfilenames:
            process(['ffmpeg-normalize', librarypath + filename, '-o', normalizedpath + filename])
            process(['ffmpeg', '-loglevel', 'error', '-i', normalizedpath + filename, '-acodec', 'pcm_s16le', '-ac', '1', '-ar', '100', '-threads', '6', '-y', wavpath + fileid + '.wav'])
            process(['ffmpeg', '-loglevel', 'error', '-i', normalizedpath + filename, '-c:a', 'aac', '-ar', '44100', '-threads', '6', '-y', audiopath + audiofilename])
        
        res.append(audiofilename)
    except Exception as e:
        print(e)

filenames = res
print(filenames)
print(len(filenames))

ERROR: Output extension mp3 does not support PCM audio. Please choose a suitable audio codec with the -c:a option.


Command '['ffmpeg-normalize', '/mnt/freesound-crawler/downloads/512389__kievinay__bass-impact-ki-01.mp3', '-o', '/mnt/freesound-crawler/processed/normalized/512389__kievinay__bass-impact-ki-01.mp3']' returned non-zero exit status 1.


ERROR: Output extension flac does not support PCM audio. Please choose a suitable audio codec with the -c:a option.


Command '['ffmpeg-normalize', '/mnt/freesound-crawler/downloads/60015__qubodup__writing-z-for-zorro-in-the-air.flac', '-o', '/mnt/freesound-crawler/processed/normalized/60015__qubodup__writing-z-for-zorro-in-the-air.flac']' returned non-zero exit status 1.


ERROR: Output extension flac does not support PCM audio. Please choose a suitable audio codec with the -c:a option.


...

ERROR: Output extension ogg does not support PCM audio. Please choose a suitable audio codec with the -c:a option.


Command '['ffmpeg-normalize', '/mnt/freesound-crawler/downloads/395362__ihitokage__hit-6.ogg', '-o', '/mnt/freesound-crawler/processed/normalized/395362__ihitokage__hit-6.ogg']' returned non-zero exit status 1.


ERROR: Output extension mp3 does not support PCM audio. Please choose a suitable audio codec with the -c:a option.


Command '['ffmpeg-normalize', '/mnt/freesound-crawler/downloads/323199__alivvie__welcome-back.mp3', '-o', '/mnt/freesound-crawler/processed/normalized/323199__alivvie__welcome-back.mp3']' returned non-zero exit status 1.
Command '['ffmpeg-normalize', '/mnt/freesound-crawler/downloads/198248__unfa__laughter-04.flac', '-o', '/mnt/freesound-crawler/processed/normalized/198248__unfa__laughter-04.flac']' returned non-zero exit status 1.
['188383.m4a', '473578.m4a', '329355.m4a', '396447.m4a', '69920.m4a', '3569.m4a', '244437.m4a', '194071.m4a', '404747.m4a', '164642.m4a', '341250.m4a', '220178.m4a', '406735.m4a', '13858.m4a', '9925.m4a', '325270.m4a', '508027.m4a', '194080.m4a', '158896.m4a', '194060.m4a', '381773.m4a', '433820.m4a', '386611.m4a', '325262.m4a', '11557.m4a', '272460.m4a', '166886.m4a', '190112.m4a', '422971.m4a', '361252.m4a', '194112.m4a', '25706.m4a', '341248.m4a', '336615.m4a', '153277.m4a', '3558.m4a', '140459.m4a', '233650.m4a', '190096.m4a', '422742.m4a', '548117.m4a',

ERROR: Output extension flac does not support PCM audio. Please choose a suitable audio codec with the -c:a option.


In [11]:
df = pd.DataFrame()
out_distances = {}
full_distances = []

for leftfilename in filenames:
    leftid = leftfilename.split('.')[0]
    rightfilenames = filenames
    
    for righti, rightfilename in enumerate(rightfilenames):
        if (len(full_distances) % 100) == 0 and righti == 0:
            print(str(len(full_distances)) + " / " + str(len(filenames) * MAX_SAMPLE_COMPARISON_COUNT))
            
        if righti >= MAX_SAMPLE_COMPARISON_COUNT:
            continue
        
        if leftfilename == rightfilename:
            continue
            
        rightid = rightfilename.split('.')[0]
            
        try:
            correlation = correlate(wavpath, leftid, rightid, 'wav', dtwpath)
            
            if math.isinf(float(correlation)):
                continue
                
            if math.isnan(float(correlation)):
                continue
            
            df.loc[leftid, rightid] = correlation
            df.loc[rightid, leftid] = correlation

            if leftid not in out_distances:
                out_distances[leftid] = []
            
            duration = samples[leftid]['duration']
            out_distances[leftid].append([correlation, duration, int(rightid)])
            full_distances.append([correlation, leftid, rightid])

        except Exception as e:
            pass

for filename in out_distances:
    out_distances[filename] = sorted(out_distances[filename])
    
full_distances = sorted(full_distances)
print(len(full_distances))

0 / 98000
0 / 98000


/home/andrus/Projects/breathic/dtw.py:28: RuntimeWarning: invalid value encountered in divide
  normalized_samples = samples / max(samples)


0 / 98000
0 / 98000
0 / 98000
0 / 98000
0 / 98000
0 / 98000
0 / 98000
0 / 98000
0 / 98000
0 / 98000


/home/andrus/Projects/breathic/dtw.py:28: RuntimeWarning: invalid value encountered in divide
  normalized_samples = samples / max(samples)
/home/andrus/Projects/breathic/dtw.py:28: RuntimeWarning: divide by zero encountered in divide
  normalized_samples = samples / max(samples)


600 / 98000
600 / 98000
600 / 98000
600 / 98000
600 / 98000
600 / 98000
600


In [12]:
delimiter = '|'
samples = {}
filtered_filenames = [f for f in os.listdir(dtwpath) if f.endswith('.dtw')]

for filename in filtered_filenames:
    filename = filename.split('.dtw')[0]
    left, right = filename.split(delimiter)
    
    if left not in samples:
        samples[left] = []
        
    if right not in samples:
        samples[right] = []
        
    samples[left].append(right)
    samples[right].append(left)

In [26]:
BREATHE_OUT_RATE = 0.5
saved = []

for distance in full_distances:
    leftid = distance[1]
    
    if leftid in saved:
        continue

    source = audiopath + leftid + '.' + audiofileformat
    target = extensionsamplepath + leftid + '.' + audiofileformat
    target_breathe_out = extensionsamplepath + leftid + '-breathe-out.' + audiofileformat
    process(['cp', source, target])
    process(['ffmpeg', '-loglevel', 'error', '-i', target, '-af', 'asetrate=44100*' + str(BREATHE_OUT_RATE) + ',aresample=44100,atempo=1/' + str(BREATHE_OUT_RATE), '-y', target_breathe_out])
    saved.append(leftid)

with open(extensionpath + 'distances.json', 'w') as fp:
    json.dump(out_distances, fp)

In [ ]:
plot = sns.heatmap(df, xticklabels=False, yticklabels=False)
handles, labels = plot.get_legend_handles_labels()
plot.legend(reversed(handles), reversed(labels), title='', loc="lower right")
fig = plot.get_figure()
fig.tight_layout()

In [ ]:
def show_audio(comparison):
    distance, leftid, rightid = comparison
    print(distance, leftid, rightid)

    data, rate = librosa.core.load(audiopath + leftid + '.' + audiofileformat)
    ipd.display(ipd.Audio(data, rate=rate))

    data, rate = librosa.core.load(audiopath + rightid + '.' + audiofileformat)
    ipd.display(ipd.Audio(data, rate=rate))

display_count = 10
for i in range(display_count):
    if (i % 2) == 0:
        show_audio(full_distances[i])
        
for i in range(display_count):
    if (i % 2) == 0:
        show_audio(full_distances[i - display_count])